In [1]:
import numpy as np
import pickle
import sys, os
sys.path.append(os.environ['rapp'])
sys.path.append(os.environ['raco'])
from common import *
labelsize = 8

In [2]:
dirnames = ['/nobackup/lmatilsk/case_H', '/nobackup/lmatilsk/case_M']
dirname = dirnames[1] # default dirname for other stuff
rbcz = [0.726, 0.729]
rov = [0.701, 0.710]

In [3]:
# get grid info
di_grid = get_grid_info(dirname)
rr = di_grid['rr']
rmin = np.min(rr)
rmax = np.max(rr)
r0 = 5e10
rw = di_grid['rw']

In [4]:
def av_over_r(prof, r1, r2):
    ir1 = np.argmin(np.abs(rr/rsun - r1))
    ir2 = np.argmin(np.abs(rr/rsun - r2))
    return np.sum( (prof*rw)[ir2:ir1+1])/np.sum(rw[ir2:ir1+1]) 

In [5]:
# define some system parameters and get background state
eq = get_eq(dirname)
H = rmax - r0
lsun = 3.846e33
om0 = 8.61e-6
cp = 3.5e8
n2 = eq.dsdr/cp*eq.gravity

In [6]:
# get shell average profiles
vals = []
lut = []

for dirn in dirnames:
    the_file = get_widest_range_file(dirn + '/data/', 'Shell_Avgs')
    print ('reading', the_file)
    di = get_dict(the_file)
    vals.append(di['vals'])
    lut.append(di['lut'])

reading /nobackup/lmatilsk/case_H/data/Shell_Avgs-02467500_18895000.pkl
reading /nobackup/lmatilsk/case_M/data/Shell_Avgs-07802500_50222500.pkl


In [7]:
# compute diagnostic numbers
Re = []
Rem = []
Ro = []
n2ratio = []

for i in range(2):
    vals_loc = vals[i]
    lut_loc = lut[i]
    rov_loc = rov[i]
    rbcz_loc = rbcz[i]
    
    rke = vals_loc[:, 0, lut_loc[410]]
    tke = vals_loc[:, 0, lut_loc[411]]
    pke = vals_loc[:, 0, lut_loc[412]]
    
    cflux = vals_loc[:, 0, lut_loc[1470]]
    dsdr_l0 = -cflux/(eq.rho*eq.T*eq.kappa)
    n2_add = dsdr_l0/cp*eq.gravity
    n2ratio_loc = (n2 + n2_add)/(om0)**2
    
    ke = rke + tke + pke
    v2 = ke/eq.rho
    v = np.sqrt(v2)
    
    r1, r2 = rbcz_loc, rmax/rsun
    Re.append( av_over_r(v, r1, r2) * H / av_over_r(eq.nu, r1, r2))
    Rem.append( av_over_r(v, r1, r2) * H / av_over_r(eq.eta, r1, r2))
    Ro.append( av_over_r(v, r1, r2) / (2*om0*H))
    n2ratio.append( av_over_r(n2ratio_loc, r1, r2))
    
    r1, r2 = rov_loc, rbcz_loc   
    Re.append( av_over_r(v, r1, r2) * H / av_over_r(eq.nu, r1, r2))
    Rem.append( av_over_r(v, r1, r2) * H / av_over_r(eq.eta, r1, r2))
    Ro.append( av_over_r(v, r1, r2) / (2*om0*H))
    n2ratio.append( av_over_r(n2ratio_loc, r1, r2))
    
    r1, r2 = rmin/rsun, rov_loc   
    Re.append( av_over_r(v, r1, r2) * H / av_over_r(eq.nu, r1, r2))
    Rem.append( av_over_r(v, r1, r2) * H / av_over_r(eq.eta, r1, r2))
    Ro.append( av_over_r(v, r1, r2) / (2*om0*H))
    n2ratio.append( av_over_r(n2ratio_loc, r1, r2))

In [8]:
Re

[45.81406832935589,
 58.44898998228546,
 15.70358821297375,
 36.31797216896867,
 21.57664780216914,
 2.862529727136516]

In [9]:
Rem

[183.25627331742356,
 233.79595992914184,
 62.814352851895,
 145.2718886758747,
 86.30659120867656,
 11.450118908546065]

In [10]:
Ro

[0.024699950334565714,
 0.014814974233724573,
 0.002590962759694133,
 0.01964872222396549,
 0.005579474937683489,
 0.0004843636027998933]

In [11]:
n2ratio

[-0.7253753523641906,
 1131.8150377549146,
 27994.659632778126,
 -0.7103419231126366,
 205.7032052735687,
 26672.37671651066]

In [12]:
def printrow(row):
    st = ''
    for member in row:
        st += '%1.6f & ' %member
    st = st[:-3]
    st += '\\\\'
    return st

print (printrow(Re))
print (printrow(Rem))
print (printrow(Ro))
print (printrow(n2ratio))

45.814068 & 58.448990 & 15.703588 & 36.317972 & 21.576648 & 2.862530\\
183.256273 & 233.795960 & 62.814353 & 145.271889 & 86.306591 & 11.450119\\
0.024700 & 0.014815 & 0.002591 & 0.019649 & 0.005579 & 0.000484\\
-0.725375 & 1131.815038 & 27994.659633 & -0.710342 & 205.703205 & 26672.376717\\


In [13]:
# compute flux Rayleigh number and Ekman number
nu_cz = av_over_r(eq.nu, r0/rsun, rmax/rsun)
F_rad = di['vals'][:, 0, di['lut'][1433]]
F_other = lsun/(4*np.pi*rr**2)
F_cz = av_over_r(F_other, r0/rsun, rmax/rsun)
g_cz = av_over_r(eq.gravity, r0/rsun, rmax/rsun)
rho_cz = av_over_r(eq.rho, r0/rsun, rmax/rsun)
T_cz = av_over_r(eq.T, r0/rsun, rmax/rsun)

In [14]:
RaF = g_cz*F_cz*H**4/(cp*rho_cz*T_cz*nu_cz**3)
RaF

749717.5643120643

In [15]:
Ek = nu_cz/om0/H**2
Ek

0.0010662306820766645

In [16]:
Di = g_cz*H/(cp*T_cz)
Di

1.7234241375520245